# ICU Clinical Handoff Summary Pipeline

EMR 데이터 로드 → 소견 추출 → 해석 (중복 제거/충돌 해소) → 인수인계 요약 생성

이 노트북은 Google Colab에서 실행할 수 있도록 설계되었습니다.

## 1. Setup & Install

In [ ]:
!git clone https://github.com/taejun-song/emr-icu-handover.git
%cd emr-icu-handover
!pip install -q anthropic pandas openpyxl pydantic python-dotenv

In [ ]:
import os
from google.colab import userdata
os.environ["ANTHROPIC_API_KEY"] = userdata.get("ANTHROPIC_API_KEY")

In [ ]:
import json
import pandas as pd
from pathlib import Path
from IPython.display import Markdown, display
from src.config import (
    BASELINE_FILE, INPUT_DATA_FILE, OUTPUT_DIR, EXTRACTIONS_DIR,
    OUTPUT_FRAMEWORK_FILE, SHEET_NAMES, SHEET_NAME_TO_PROMPT,
)
from src.loader import load_emr_file
from src.extractors import extract_all, extract_sheet
from src.interpreter import interpret
from src.synthesizer import synthesize
from src.schemas import ExtractorOutput

## 2. Load EMR Data

In [ ]:
baseline_sheets = load_emr_file(BASELINE_FILE)
data_sheets = load_emr_file(INPUT_DATA_FILE)

print(f"Baseline sheets: {list(baseline_sheets.keys())}")
print(f"Data sheets: {list(data_sheets.keys())}")
for name, df in data_sheets.items():
    print(f"  {name}: {len(df)} rows")

## 3. Run Extractors

In [ ]:
extractor_outputs = await extract_all(data_sheets)

print(f"Extracted from {len(extractor_outputs)} sheets:")
for eo in extractor_outputs:
    print(f"  {eo.sheet_name}: {len(eo.findings)} findings")

In [ ]:
EXTRACTIONS_DIR.mkdir(parents=True, exist_ok=True)
for eo in extractor_outputs:
    out_path = EXTRACTIONS_DIR / f"{eo.sheet_name.replace(' ', '_').lower()}.json"
    out_path.write_text(json.dumps(eo.model_dump(), ensure_ascii=False, indent=2), encoding="utf-8")
    print(f"Saved: {out_path.name}")

## 4. Extraction Inspection

In [ ]:
results = {eo.sheet_name: eo for eo in extractor_outputs}

for sheet_name, eo in results.items():
    source_rows = len(data_sheets[sheet_name])
    findings_count = len(eo.findings)
    header = (
        f"### {sheet_name}\n\n"
        f"**Source rows:** {source_rows} | "
        f"**Findings extracted:** {findings_count}\n\n"
    )
    rows = []
    for i, f in enumerate(eo.findings, 1):
        rows.append(f"| {i} | {f.datetime or '—'} | {f.category} | {f.content[:120]}{'…' if len(f.content) > 120 else ''} |")
    table = "| # | Datetime | Category | Content |\n|---|----------|----------|---------|\n" + "\n".join(rows)
    display(Markdown(header + table))
    display(Markdown("---"))

### Coverage Analysis

In [ ]:
coverage_rows = []
for sheet_name in SHEET_NAMES:
    source_rows = len(data_sheets.get(sheet_name, pd.DataFrame()))
    findings = len(results[sheet_name].findings) if sheet_name in results else 0
    pct = (findings / source_rows * 100) if source_rows > 0 else 0.0
    coverage_rows.append({"Sheet": sheet_name, "Source Rows": source_rows, "Findings Extracted": findings, "Coverage %": round(pct, 1)})

coverage_df = pd.DataFrame(coverage_rows)
display(coverage_df)

## 5. Run Interpreter

In [ ]:
interpreter_output = await interpret(extractor_outputs, baseline_sheets)

print(f"Reconciled findings: {len(interpreter_output.reconciled_findings)}")
print(f"Conflicts resolved:  {len(interpreter_output.conflicts_resolved)}")
print(f"Duplicates removed:  {interpreter_output.duplicates_removed}")
print(f"Input findings:      {interpreter_output.metadata.total_input_findings}")
print(f"Output findings:     {interpreter_output.metadata.total_output_findings}")

In [ ]:
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)
interp_path = OUTPUT_DIR / "interpretation.json"
interp_path.write_text(
    json.dumps(interpreter_output.model_dump(), ensure_ascii=False, indent=2),
    encoding="utf-8",
)
print(f"Saved: {interp_path}")

## 6. Interpretation Inspection

In [ ]:
rows = []
for i, rf in enumerate(interpreter_output.reconciled_findings, 1):
    sources = ", ".join(rf.sources)
    note = rf.resolution_note or "—"
    content_preview = rf.content[:100] + ("…" if len(rf.content) > 100 else "")
    rows.append(f"| {i} | {rf.datetime or '—'} | {content_preview} | {sources} | {note} |")

table = (
    "| # | Datetime | Content | Sources | Resolution Note |\n"
    "|---|----------|---------|---------|--------------------|\n"
    + "\n".join(rows)
)
display(Markdown(table))

### Conflict Resolutions

In [ ]:
if not interpreter_output.conflicts_resolved:
    display(Markdown("*No conflicts detected.*"))
else:
    for i, cr in enumerate(interpreter_output.conflicts_resolved, 1):
        md = (
            f"### Conflict {i}\n\n"
            f"**Description:** {cr.description}\n\n"
            f"**Sources:** {', '.join(cr.sources)}\n\n"
            f"**Resolution:** {cr.resolution}"
        )
        display(Markdown(md))

## 7. Run Synthesizer

In [ ]:
synthesizer_output = await synthesize(interpreter_output)

print(f"Findings incorporated: {synthesizer_output.metadata.findings_incorporated}")
print(f"Date range: {synthesizer_output.metadata.date_range}")

## 8. Synthesis Inspection

In [ ]:
framework_sheets = pd.read_excel(OUTPUT_FRAMEWORK_FILE, engine="openpyxl", sheet_name=None)
expected_sections = list(framework_sheets.keys())

summary_text = synthesizer_output.summary.lower()
coverage_rows = []
for section in expected_sections:
    found = section.lower() in summary_text
    coverage_rows.append({"Section": section, "Found in Summary": found})

coverage_df = pd.DataFrame(coverage_rows)
found_count = coverage_df["Found in Summary"].sum()
total = len(coverage_df)
print(f"Section coverage: {found_count}/{total} ({found_count/total*100:.0f}%)\n")
display(coverage_df)

## 9. Final Summary

In [ ]:
display(Markdown(synthesizer_output.summary))

In [ ]:
summary_path = OUTPUT_DIR / "summary.md"
summary_path.write_text(synthesizer_output.summary, encoding="utf-8")
print(f"Saved: {summary_path}")